In [1]:
import numpy as np
import pandas as pd
import os, sys
from datetime import timedelta
import torch

sys.path.append(os.path.join(root_dir, 'bogovirus/causica'))
sys.path.append(os.path.join(root_dir, 'bogovirus'))

%load_ext autoreload
%autoreload 2

In this notebook we use Causica (DECI) model for End-to-End Causal Inference. Therefore, to run the notebook, you would need to first to clone causica package from the following link to the current directory:
- https://github.com/microsoft/causica/tree/main/causica

In [2]:
# pip install pyvis
# pip install networkx
# pip install openpyxl

### Data Preparation
We read the data from BogoVirus with random dose policy. We then convert the data in a format containing information of prev. and cur. days. Patients are sampled into train-test files. All the records of the the patients in trainset is used to train DECI model. The test set is used for validation of ATE estimations, which is not covered in this notebook.

In [3]:
def extend_features_with_temporal_window(df_orig, id_columns, to_extend_columns, temp_start=1, temp_step_size=1, temp_num_steps=1):
    """
    temporal_start: what is the temporal distance of first temporal feature (prev & next). Default: 1
    (e.g., temporal_start=1 adds X_{t-1} & X_{t+1} or temporal_start=2 adds X_{t-2} & X_{t+2} to the featuresets)
    temp_step_size: the distance of two temporal features on each side of the feature; if the temp_num_steps=0, then this value does not matter. Default: 1
    temp_num_steps: the number of temporal features on each side. Default: 1
    
    If you want to include temporal features in multiple distances, you can change the temp_step_size & temp_num_steps to control the step size and the number of steps. For example if temp_start=1, temp_step_size=2, temp_num_steps=3, it will add X_{t-1} & X_{t+1} features to the features and then since temp_step_size equals to 2, it adds X_{t-3} & X_{t+3} to the featureset. As the temp_num_steps=3, it also adds X_{t-5} & X_{t+5}, which adds up to three temporal features on each side.
    """
    
    df_ext = df_orig.copy()
    colnames = list(df_ext.columns)
    
    for step in range(temp_num_steps):
        #shift values down by 1 => all cols but time
        df_tprev = df_ext[id_columns+to_extend_columns].groupby(id_columns).shift(temp_start + step*temp_step_size)[len(id_colname):] #shift down
        # df_tnext = df_ext[id_columns+to_extend_columns].groupby(id_columns).shift(-(temp_start + step*temp_step_size))[len(id_colname):] #shift up

        # rename columns
        df_tprev.columns = ['prev' +str(temp_start + (step)*temp_step_size)+'_' + tagname for tagname in df_tprev.columns]
        # df_tnext.columns = ['next' +str(temp_start + (step)*temp_step_size)+'_' + tagname for tagname in df_tnext.columns]

        # column-names correction
        colnames = colnames + list(df_tprev.columns)
        # colnames = colnames + list(df_tnext.columns)

        # combine starting from original => then prev => then next
        # df_ext = pd.concat([df_ext, df_tprev, df_tnext], axis=1)
        df_ext = pd.concat([df_ext, df_tprev], axis=1)
    return df_ext, colnames

In [4]:
# data directory
data_dir = 'bogovirus/data/'
df = pd.read_csv(data_dir + 'patient_data_random_dose.csv')
df.head(2)

,patient_id,cohort,day_number,infection_prev,severity,cum_drug_prev,infection,drug,cum_drug,severity_next,toxicity,outcome
0,1243,0,5,61,53.122915,0.2880,67,1.0,0.57280,52.419031,0.035320,none
1,1243,0,6,67,52.419031,0.5728,74,1.0,0.74368,53.590940,0.169167,none


In [8]:
# cat_to_binary
df["die"]=(df["outcome"]=="die").astype(int)
df["recover"]=(df["outcome"]=="recover").astype(int)
df_ground = df.copy()

# drop cohort info, too
df=df.drop(columns=["cohort"])
df.head(2)

,patient_id,day_number,infection_prev,severity,cum_drug_prev,infection,drug,cum_drug,severity_next,toxicity,outcome,die,recover
0,1243,5,61,53.122915,0.2880,67,1.0,0.57280,52.419031,0.035320,none,0,0
1,1243,6,67,52.419031,0.5728,74,1.0,0.74368,53.590940,0.169167,none,0,0


In [9]:
base_colnames=["severity", "infection", "drug", "cum_drug", "toxicity", "die", "recover"]
prev_colnames=["infection_prev", "cum_drug_prev"]

time_colname="day_number"
id_colname="patient_id"

df=df[[id_colname,time_colname]+base_colnames+prev_colnames].sort_values(by=[id_colname,time_colname])
df, colnames=extend_features_with_temporal_window(df_orig=df, id_columns=[id_colname],to_extend_columns=base_colnames+prev_colnames, temp_start=1, temp_step_size=1, temp_num_steps=1)

df["prev1_cum_drug"]=df["cum_drug_prev"]
df["prev1_infection"]=df["infection_prev"]

In [10]:
df.loc[df.day_number==0, "prev1_drug"]=0
df.loc[df.day_number==0, "prev1_die"]=0
df.loc[df.day_number==0, "prev1_recover"]=0
df.loc[df.day_number==0, "prev1_toxicity"]=0

df = df.drop(columns=["cum_drug_prev", "infection_prev", "prev1_infection_prev", "prev1_cum_drug_prev"])
df_day0 = df[df.day_number==0]

df.head(2)

,patient_id,day_number,severity,infection,drug,cum_drug,toxicity,die,recover,prev1_severity,prev1_infection,prev1_drug,prev1_cum_drug,prev1_toxicity,prev1_die,prev1_recover
0,1243,5,53.122915,67,1.0,0.57280,0.035320,0,0,46.983000,61,0.3,0.2880,0.000571,0.0,0.0
1,1243,6,52.419031,74,1.0,0.74368,0.169167,0,0,53.122915,67,1.0,0.5728,0.035320,0.0,0.0


In [12]:
# cohort
patient_cohort = list(set(df_day0.patient_id))
patient_cohort_seq = df.groupby("patient_id")["day_number"].count()

# train - test 
import random
random.seed(42)
random.shuffle(patient_cohort)
train_point= int(0.8*len(patient_cohort))
train_patient = patient_cohort[:train_point]
test_patient = patient_cohort[train_point:]

df_train = df[df["patient_id"].isin(train_patient)].sample(frac=1.0, replace=True, random_state=42)
df_test = df[df["patient_id"].isin(test_patient)]

In [13]:
df_all = df.drop(columns=[id_colname, time_colname])
df_train_cohort = df_train.drop(columns=[id_colname, time_colname])
df_test_cohort = df_test.drop(columns=[id_colname, time_colname])

In [14]:
from helpers.deci_utils import split_train_test, create_variables_json
target_cols=['die','recover']

# This data pre-processing treat all variables as continuous, although some of them are ordinal --  categorical_vars={'status':3}
variables_json_path=data_dir + 'bogo_time/variables.json'
create_variables_json(df_all, categorical_vars={'die':2, 'recover':2,'prev1_die':2, 'prev1_recover':2}, variables_json_path=variables_json_path, text_vars=None, target_column_names=[])

print("DECI internal trainsize: ", int(0.8 * df_train_cohort.shape[0]))
print("DECI internal val & test size: ", int(0.1 * df_train_cohort.shape[0]))
print("Leave-out size: ", df_test_cohort.shape[0])

DECI internal trainsize:  90002
DECI internal val & test size:  11250
Leave-out size:  28244


### DECI training with no graph constraints

In [15]:
from datetime import datetime
from helpers.deci_utils import run_deci_train

device = "0" if torch.cuda.is_available() else "cpu"
model_name = f"bogo_{datetime.now():%Y_%m_%d_%H%M%S}"
experiment_dir = "bogovirus/runs/" + model_name

model, networkx_graph, variable_name_dict = run_deci_train(df_train_cohort, model_name, experiment_dir, variables_json_path=variables_json_path, device=device)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/matavako1-bwh-gpu/code/Users/matavako/bogovirus/causica/causica/utils/helper_functions.py:35: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  return tuple(torch.as_tensor(array, dtype=dtype, device=device) for array in arrays)


Saving logs to bogovirus/runs/bogo_2022_09_29_082120/train_output/summary
Auglag Step: 0
LR: 0.001
Inner Step: 100, loss: 4875.97, log p(x|A): -4875.97, dag: 5.37892575, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 0.000222, cts_mse_icgnn: 1.06e+04
Inner Step: 200, loss: 2973.41, log p(x|A): -2973.41, dag: 5.20889270, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 0.000188, cts_mse_icgnn: 7.61e+03
Inner Step: 300, loss: 1772.47, log p(x|A): -1772.47, dag: 6.26098974, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 0.000237, cts_mse_icgnn: 5.11e+03
Inner Step: 400, loss: 1062.35, log p(x|A): -1062.35, dag: 6.76904315, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 0.000276, cts_mse_icgnn: 3.33e+03
Inner Step: 500, loss: 714.19, log p(x|A): -714.19, dag: 7.94256885, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 

In [16]:
print("Saved as ", model_name)

Saved as  bogo_2022_09_29_082120


In [17]:
from helpers.deci_utils import save_deci_unweighted_graph, calc_weighted_graph, save_edges_to_excel, save_deci_weighted_graph
save_deci_unweighted_graph(model, variable_name_dict, experiment_dir)
variables_json_path=data_dir + 'bogo_time/variables.json'
deci_weighted_graph = calc_weighted_graph(model, df_train_cohort, variables_json_path, experiment_dir, samples=200)
save_edges_to_excel(deci_weighted_graph, variable_name_dict, experiment_dir)
save_deci_weighted_graph(deci_weighted_graph, variable_name_dict, experiment_dir, conf_threshold=0.7)

(112503, 14)
Computing the ATE between X0=66.63080499590015 and X0=21.171204406654127
Computing the ATE between X1=83.65770222849277 and X1=43.033674890339626
Computing the ATE between X2=0.9650484550193252 and X2=-0.0008377228610756959
Computing the ATE between X3=0.6710106609600918 and X3=0.12625069568653213
Computing the ATE between X4=0.2507789693686753 and X4=-0.10635255473245545
Computing the ATE between X5=0.2541000561380689 and X5=-0.16297359728808267
Computing the ATE between X6=0.18157169811677804 and X6=-0.13135081511810243
Computing the ATE between X7=61.37229545540491 and X7=20.94418525459091
Computing the ATE between X8=78.94147529401741 and X8=38.73424890889273
Computing the ATE between X9=0.8719425218972884 and X9=-0.05258326718150541
Computing the ATE between X10=0.6056452545272621 and X10=0.08031651844486448
Computing the ATE between X11=0.13039248502693201 and X11=-0.05094388799739752
Computing the ATE between X12=0.0 and X12=0.0
Computing the ATE between X13=0.0 and

In [21]:
df_train_day0 = df_train[df_train.day_number==0][df_train_cohort.columns]
df_train_day0.head()

,severity,infection,drug,cum_drug,toxicity,die,recover,prev1_severity,prev1_infection,prev1_drug,prev1_cum_drug,prev1_toxicity,prev1_die,prev1_recover
129560,23.0,28,0.0,0.00,0.000000e+00,0,0,NaN,20,0.0,0.0,0.0,0.0,0.0
7910,17.0,34,0.0,0.00,0.000000e+00,0,0,NaN,34,0.0,0.0,0.0,0.0,0.0
82098,26.0,32,0.0,0.00,0.000000e+00,0,0,NaN,24,0.0,0.0,0.0,0.0,0.0
84918,14.0,27,0.0,0.00,0.000000e+00,0,0,NaN,25,0.0,0.0,0.0,0.0,0.0
46870,14.0,46,0.2,0.08,2.621440e-07,0,0,NaN,38,0.0,0.0,0.0,0.0,0.0


In [29]:
from scipy import stats
import os
os.mkdir(os.path.join(experiment_dir, 'simsim_data'))

### Adding domain-specific graph constraints
To improve the quality of the learned graph, it is possible to place constraints on the graph that DECI learns. The constraints come in two flavours:
 - *negative constraints* mean a certain edge cannot exist in the graph,
 - *positive constraints* mean a certain edge must exist in the graph.

In [30]:
# Constraint matrix coding is described in the DECI docstrings
# The constraint matrix has the same shape as the adjacency matrix
# A `nan` value means no constraint
# A 0 value means a negative constraint
# A 1 value means a positive constraint

constraint_matrix = np.full((len(variable_name_dict.keys()), len(variable_name_dict.keys())), np.nan, dtype=np.float32)

In [31]:
model.networkx_graph().edges

OutEdgeView([(0, 1), (0, 7), (1, 8), (2, 0), (2, 4), (2, 5), (3, 8), (4, 0), (4, 3), (4, 5), (5, 0), (5, 6), (7, 1), (8, 6), (9, 0), (9, 1), (9, 7), (9, 8), (9, 10), (10, 1), (10, 2), (10, 4), (10, 7), (11, 0), (11, 1), (11, 2), (11, 3), (11, 5), (11, 9), (11, 10), (12, 1), (12, 5), (12, 7)])

Temporal: First, we add temporal constraints, future does not cause the past

In [32]:
# temporal constraints
prev_idxes = [dataset.variables.name_to_idx[var] for var in variable_name_dict.values() if var.startswith("prev1")]
cur_idxes = [dataset.variables.name_to_idx[var] for var in variable_name_dict.values() if not var.startswith("prev1")]

for prev_idx in prev_idxes:
    constraint_matrix[cur_idxes, prev_idx] = 0.

Secondly, constraints on the outcomes not to be a cause for others

In [33]:
outcome_factors = ['die', 'recover']
outcome_idxes = [dataset.variables.name_to_idx[var] for var in outcome_factors]
nonoutcome_idxes = [dataset.variables.name_to_idx[var] for var in variable_name_dict.values() if var not in outcome_factors]

for out_idx in outcome_idxes:
    constraint_matrix[out_idx, nonoutcome_idxes] = 0.

Thirdly, we know severity is a cause for sevirity_next, infection_prev to infection_next, and drug to cum_drug

In [34]:
constraint_matrix[dataset.variables.name_to_idx['severity'], dataset.variables.name_to_idx['drug']]=1.0
constraint_matrix[dataset.variables.name_to_idx['drug'], dataset.variables.name_to_idx['severity']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_severity'], dataset.variables.name_to_idx['severity']]=1.0
constraint_matrix[dataset.variables.name_to_idx['prev1_infection'], dataset.variables.name_to_idx['infection']]=1.0

constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['severity']]=1.0
constraint_matrix[dataset.variables.name_to_idx['prev1_cum_drug'], dataset.variables.name_to_idx['severity']]=1.0
constraint_matrix[dataset.variables.name_to_idx['drug'], dataset.variables.name_to_idx['cum_drug']]=1.0
constraint_matrix[dataset.variables.name_to_idx['cum_drug'], dataset.variables.name_to_idx['drug']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['prev1_cum_drug']]=1.0
constraint_matrix[dataset.variables.name_to_idx['prev1_cum_drug'], dataset.variables.name_to_idx['prev1_drug']]=0.0

In [35]:
df_train_cohort.head()

,severity,infection,drug,cum_drug,toxicity,die,recover,prev1_severity,prev1_infection,prev1_drug,prev1_cum_drug,prev1_toxicity,prev1_die,prev1_recover
19903,54.607584,55,0.8,0.479744,1.219150e-02,0,0,47.696857,53,0.2,0.266240,3.561556e-04,0.0,0.0
1173,46.535000,75,1.0,0.857256,3.968847e-01,0,0,50.113805,73,1.4,0.762094,1.959076e-01,0.0,0.0
129560,23.000000,28,0.0,0.000000,0.000000e+00,0,0,NaN,20,0.0,0.000000,0.000000e+00,0.0,0.0
137721,44.329172,51,0.5,0.234560,1.665420e-04,0,0,35.753793,50,0.0,0.057600,3.652035e-08,0.0,0.0
96872,33.428911,70,0.0,0.062208,5.795320e-08,0,0,24.480828,65,0.0,0.103680,1.242138e-06,0.0,0.0


In [36]:
from helpers.deci_utils import run_deci_train_with_constraints
model_name_c = model_name + '_c'
experiment_dir_c = experiment_dir + '_c'
model_c, networkx_graph_c, variable_name_dict = run_deci_train_with_constraints(
    df_train_cohort, constraint_matrix, model_name_c, experiment_dir_c, variables_json_path, device)

Saving logs to bogovirus/runs/bogo_2022_09_29_082120_c/train_output/summary
Auglag Step: 0
LR: 0.001
Inner Step: 100, loss: 4857.22, log p(x|A): -4857.22, dag: 1.24782837, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.49e-05, cts_mse_icgnn: 1.06e+04
Inner Step: 200, loss: 2962.22, log p(x|A): -2962.22, dag: 1.21755986, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.42e-05, cts_mse_icgnn: 7.58e+03
Inner Step: 300, loss: 1774.74, log p(x|A): -1774.74, dag: 1.54822399, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.89e-05, cts_mse_icgnn: 5.12e+03
Inner Step: 400, loss: 1064.54, log p(x|A): -1064.54, dag: 1.24019377, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.22e-05, cts_mse_icgnn: 3.33e+03
Inner Step: 500, loss: 680.25, log p(x|A): -680.25, dag: 1.62380771, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec

In [37]:
from helpers.deci_utils import save_deci_unweighted_graph, calc_weighted_graph, save_edges_to_excel, save_deci_weighted_graph
save_deci_unweighted_graph(model_c, variable_name_dict, experiment_dir_c)
variables_json_path=data_dir + 'bogo_time/variables.json'
deci_weighted_graph = calc_weighted_graph(model_c, df_train_cohort, variables_json_path, experiment_dir_c, samples=200)
save_edges_to_excel(deci_weighted_graph, variable_name_dict, experiment_dir_c)
save_deci_weighted_graph(deci_weighted_graph, variable_name_dict, experiment_dir_c, conf_threshold=0.7)

(112503, 14)
Computing the ATE between X0=66.63080499590015 and X0=21.171204406654127
Computing the ATE between X1=83.65770222849277 and X1=43.033674890339626
Computing the ATE between X2=0.9650484550193252 and X2=-0.0008377228610756959
Computing the ATE between X3=0.6710106609600918 and X3=0.12625069568653213
Computing the ATE between X4=0.2507789693686753 and X4=-0.10635255473245545
Computing the ATE between X5=0.2541000561380689 and X5=-0.16297359728808267
Computing the ATE between X6=0.18157169811677804 and X6=-0.13135081511810243
Computing the ATE between X7=61.37229545540491 and X7=20.94418525459091
Computing the ATE between X8=78.94147529401741 and X8=38.73424890889273
Computing the ATE between X9=0.8719425218972884 and X9=-0.05258326718150541
Computing the ATE between X10=0.6056452545272621 and X10=0.08031651844486448
Computing the ATE between X11=0.13039248502693201 and X11=-0.05094388799739752
Computing the ATE between X12=0.0 and X12=0.0
Computing the ATE between X13=0.0 and

# Simulation
DECI has also fitted an SCM that captures the functional relationship and error distribution of this dataset. We can estimate ATE and CATE using do-based sampling E\[X_{today}|do(X_{yesterday},drug=dose)].

## Treatment effect estimation -- Definitions

Treatment effects describe how a target variable (such as outcome_die) changes in response to an intervention on some other variables.
 - **Average Treatment Effect (ATE)** gives the difference between giving the drug to every patient, and not giving the treatment to any of them. This summarizes the overall or average effectiveness of the drug.
 - **Conditional Average Treatment Effect (CATE)** gives the difference between giving and not giving the drug, *restricted to a certain conditions or subgroup of patients*. For example, we could investigate the average treatment effect effect of drug on sever or nonsever patients (e.g., severity_now < 30. vs. severity_now > 109),.
 - **Individual Treatment Effect (ITE)** is the different in outcome_die between giving and not giving the drug *for one specific patient*.

In [ ]:
from helpers.deci_utils import get_dataset_variables
dataset, variables = get_dataset_variables(df_train_cohort, variables_json_path)

In [23]:
categorical_vars=['die', 'recover','prev1_die', 'prev1_recover']
categorical_vars_idx=[dataset.variables.name_to_idx[v] for v in categorical_vars]
categorical_vars_idx


[5, 6, 12, 13]

In [ ]:
import random
def is_out(row):
    return (row[
        "die"] > 0.9 or row[
        "recover"] > 0.9 or row[
        "prev1_die"] > 0.9 or row[
        "prev1_recover"] > 0.9)
    return False
    
def get_drug(row, dose):
    # depends on: severity
    # treatment threshold for today
    tx_threshold = random.randint(10, 50)
    drug = dose if row['severity'] > tx_threshold else 0
    return drug


def get_next_day(model, prev_day, colnames, drug_dose=None, sample_count=1):
    cat_vars=[5, 6, 12, 13] #die/recover

    intervention_idxs=[]
    intervention_values=[]

    vars = [col for col in colnames if col.startswith('prev1_')]
    
    intervention_idxs += [dataset.variables.name_to_idx[var] for var in vars]
    intervention_values += [prev_day[var[len("prev1_"):]] for var in vars]

    ### Model-based ATE estimate
    next_day = model.sample(
        sample_count, intervention_idxs=np.array(intervention_idxs), 
        intervention_values=np.array(intervention_values)
    ).cpu().numpy()
    next_day_agg = next_day.mean(axis=0)

    tx_threshold = random.randint(10, 50)
    if (drug_dose is not None) and (next_day_agg[0] > tx_threshold): #severity
        # drug dose
        intervention_idxs = [dataset.variables.name_to_idx[
            "severity"], dataset.variables.name_to_idx["drug"]] + intervention_idxs
        intervention_values = [next_day_agg[0], drug_dose] + intervention_values

        next_day = model.sample(
                sample_count, intervention_idxs=np.array(intervention_idxs), 
                intervention_values=np.array(intervention_values)
            ).cpu().numpy()

    next_day_agg = next_day.mean(axis=0)
    next_day_agg[cat_vars] = stats.mode(next_day[:,cat_vars], axis=0).mode[0]
    return next_day_agg # next_day.mean(axis=0)

In [38]:
def run_simsim(model, population, df_core, df_prev, num_of_days, dose):
    # init
    df_none=df_core.copy()
    end_flags = [False] * population # dead or recovered

    # foreach day
    for k in range(1, num_of_days+1):
        np_simk=np.empty_like(df_prev)
        np_simk[:]=np.NaN

        # foreach patient
        for row_idx in range(np_simk.shape[0]):
            row = df_prev.iloc[row_idx,:]
            if end_flags[row_idx]:
                continue
            elif is_out(row):
                # dead or recovered
                # print('P', str(row_idx), 'gone')
                end_flags[row_idx] = True
                continue

            next_day = get_next_day(model, row, colnames=df_prev.columns, drug_dose=dose, sample_count=50)
            # print(next_day)
            np_simk[row_idx,:]=next_day
        
        df_prev=pd.DataFrame(np_simk, columns=df_prev.columns)
        # add day_number & patient_id
        df_dayk=df_prev.copy()
        # df_dayk.loc[:,df_prev.columns]=df_prev.loc[:,df_prev.columns].copy()
        df_dayk["day_number"]= [k] * population
        df_dayk["patient_id"]= df_core["patient_id"].values

        # add to the db
        df_none=df_none.append(df_dayk[df_none.columns], ignore_index=True)
        alive = len(end_flags) - sum(end_flags)

        if alive == 0:
            break
        print("Day", k, "is done, alive: ", alive)
        if k%10 == 0:
            df_none.to_csv(os.path.join(experiment_dir_c, 'simsim_data',str(k)+'.csv'))# No drug
    return df_none

In [39]:
os.mkdir(os.path.join(experiment_dir_c, 'simsim_data'))

In [165]:
population=5000 #everyone
df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() #.sample(population).copy()

#  setup
df_prev = df_core[df_train_cohort.columns] # sample instead
num_of_days=60
dose = None

df_none_c=run_simsim(model_c, population, df_core, df_prev, num_of_days, dose)

Day 1 is done, alive:  5000
Day 2 is done, alive:  5000
Day 3 is done, alive:  5000
Day 4 is done, alive:  5000
Day 5 is done, alive:  5000
Day 6 is done, alive:  5000
Day 7 is done, alive:  5000
Day 8 is done, alive:  4998
Day 9 is done, alive:  4982
Day 10 is done, alive:  4960
Day 11 is done, alive:  4918
Day 12 is done, alive:  4667
Day 13 is done, alive:  3974
Day 14 is done, alive:  2980
Day 15 is done, alive:  1905
Day 16 is done, alive:  1043
Day 17 is done, alive:  475
Day 18 is done, alive:  154
Day 19 is done, alive:  26
Day 20 is done, alive:  2


In [167]:
df_none_c.to_csv(os.path.join(experiment_dir, 'simsim_data', 'deci_constrainted_nodose-condition_max60days' +'.csv'))


In [168]:
df_none_c['die'].sum()

4299.0

In [170]:
df_none_c['recover'].sum()


701.0

Condition on a particular dose -- here dose=0.2; we do the same for all the dose candidate to find the optimal dose policy:

In [40]:
population=5000 #everyone
df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() 

#  setup
df_prev = df_core[df_train_cohort.columns] # sample instead
num_of_days=60
dose = 0.2

df02_c=run_simsim(model_c, population, df_core, df_prev, num_of_days, dose)
df02_c.to_csv(os.path.join(experiment_dir, 'simsim_data', 'deci_constrainted_dose0.2_max60days' +'.csv'))

print(df02_c['die'].sum())
print(df02_c['recover'].sum())


Day 1 is done, alive:  5000
Day 2 is done, alive:  5000
Day 3 is done, alive:  5000
Day 4 is done, alive:  5000
Day 5 is done, alive:  5000
Day 6 is done, alive:  5000
Day 7 is done, alive:  5000
Day 8 is done, alive:  5000
Day 9 is done, alive:  5000
Day 10 is done, alive:  5000
Day 11 is done, alive:  4999
Day 12 is done, alive:  4932
Day 13 is done, alive:  4532
Day 14 is done, alive:  3785
Day 15 is done, alive:  2639
Day 16 is done, alive:  1419
Day 17 is done, alive:  554
Day 18 is done, alive:  124
Day 19 is done, alive:  17
Day 20 is done, alive:  2
4779.0
221.0


Add more constraints based on the known mistakes:

In [51]:
constraint_matrix[dataset.variables.name_to_idx['prev1_severity'], dataset.variables.name_to_idx['prev1_infection']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['prev1_infection']]=0.0

constraint_matrix[dataset.variables.name_to_idx['infection'], dataset.variables.name_to_idx['severity']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_infection'], dataset.variables.name_to_idx['severity']]=1.0

constraint_matrix[dataset.variables.name_to_idx['prev1_toxicity'], dataset.variables.name_to_idx['severity']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_toxicity'], dataset.variables.name_to_idx['prev1_infection']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_toxicity'], dataset.variables.name_to_idx['drug']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_toxicity'], dataset.variables.name_to_idx['prev1_severity']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_toxicity'], dataset.variables.name_to_idx['cum_drug']]=0.0

constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['drug']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['infection']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['prev1_severity']]=0.0
constraint_matrix[dataset.variables.name_to_idx['prev1_severity'], dataset.variables.name_to_idx['prev1_drug']]=1.0

In [52]:
from helpers.deci_utils import run_deci_train_with_constraints
model_name_c2 = model_name + '_c2'
experiment_dir_c2 = experiment_dir + '_c2'
model_c2, networkx_graph_c2, variable_name_dict = run_deci_train_with_constraints(
    df_train_cohort, constraint_matrix, model_name_c2, experiment_dir_c2, variables_json_path, device)

Saving logs to bogovirus/runs/bogo_2022_09_29_082120_c2/train_output/summary
Auglag Step: 0
LR: 0.001
Inner Step: 100, loss: 4856.85, log p(x|A): -4856.85, dag: 1.03697198, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.22e-05, cts_mse_icgnn: 1.06e+04
Inner Step: 200, loss: 2958.55, log p(x|A): -2958.55, dag: 1.09737138, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.27e-05, cts_mse_icgnn: 7.57e+03
Inner Step: 300, loss: 1771.08, log p(x|A): -1771.08, dag: 1.41986729, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.59e-05, cts_mse_icgnn: 5.11e+03
Inner Step: 400, loss: 1062.12, log p(x|A): -1062.12, dag: 1.14489630, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 1.05e-05, cts_mse_icgnn: 3.33e+03
Inner Step: 500, loss: 673.04, log p(x|A): -673.04, dag: 1.29654685, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, re

In [53]:
model_name_c2

'bogo_2022_09_29_082120_c2'

In [54]:
from helpers.deci_utils import save_deci_unweighted_graph, calc_weighted_graph, save_edges_to_excel, save_deci_weighted_graph

save_deci_unweighted_graph(model_c2, variable_name_dict, experiment_dir_c2)
deci_weighted_graph_c2 = calc_weighted_graph(model_c2, df_train_cohort, variables_json_path, experiment_dir_c2, samples=200)
save_edges_to_excel(deci_weighted_graph_c2, variable_name_dict, experiment_dir_c2)
save_deci_weighted_graph(deci_weighted_graph_c2, variable_name_dict, experiment_dir_c2, conf_threshold=0.7)


(112503, 14)
Computing the ATE between X0=66.63080499590015 and X0=21.171204406654127
Computing the ATE between X1=83.65770222849277 and X1=43.033674890339626
Computing the ATE between X2=0.9650484550193252 and X2=-0.0008377228610756959
Computing the ATE between X3=0.6710106609600918 and X3=0.12625069568653213
Computing the ATE between X4=0.2507789693686753 and X4=-0.10635255473245545
Computing the ATE between X5=0.2541000561380689 and X5=-0.16297359728808267
Computing the ATE between X6=0.18157169811677804 and X6=-0.13135081511810243
Computing the ATE between X7=61.37229545540491 and X7=20.94418525459091
Computing the ATE between X8=78.94147529401741 and X8=38.73424890889273
Computing the ATE between X9=0.8719425218972884 and X9=-0.05258326718150541
Computing the ATE between X10=0.6056452545272621 and X10=0.08031651844486448
Computing the ATE between X11=0.13039248502693201 and X11=-0.05094388799739752
Computing the ATE between X12=0.0 and X12=0.0
Computing the ATE between X13=0.0 and

In [55]:
os.mkdir(os.path.join(experiment_dir_c2, 'simsim_data'))

In [56]:
population=5000 #everyone
df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() 

#  setup
df_prev = df_core[df_train_cohort.columns] # sample instead
num_of_days=60
dose = 1.4

df14_c2=run_simsim(model_c2, population, df_core, df_prev, num_of_days, dose)
df14_c2.to_csv(os.path.join(experiment_dir_c2, 'simsim_data', 'deci_constrainted_dose1.4_max60days' +'.csv'))

print(df14_c2['die'].sum())
print(df14_c2['recover'].sum())

Day 1 is done, alive:  5000
Day 2 is done, alive:  5000
Day 3 is done, alive:  4996
Day 4 is done, alive:  4950
Day 5 is done, alive:  4782
Day 6 is done, alive:  4540
Day 7 is done, alive:  4220
Day 8 is done, alive:  3806
Day 9 is done, alive:  3319
Day 10 is done, alive:  2788
Day 11 is done, alive:  2126
Day 12 is done, alive:  1496
Day 13 is done, alive:  938
Day 14 is done, alive:  532
Day 15 is done, alive:  232
Day 16 is done, alive:  95
Day 17 is done, alive:  35
Day 18 is done, alive:  8
Day 19 is done, alive:  1
4967.0
33.0


# Confounding by indication bias: drug -> severity
If the causal graph had a mistake on the direction of drug/severity, how the outcome would have been different.

In [234]:
constraint_matrix[dataset.variables.name_to_idx['severity'], dataset.variables.name_to_idx['drug']]=0.0
constraint_matrix[dataset.variables.name_to_idx['drug'], dataset.variables.name_to_idx['severity']]=1.0

constraint_matrix[dataset.variables.name_to_idx['prev1_drug'], dataset.variables.name_to_idx['prev1_severity']]=1.0
constraint_matrix[dataset.variables.name_to_idx['prev1_severity'], dataset.variables.name_to_idx['prev1_drug']]=0.0

In [235]:
from helpers.deci_utils import run_deci_train_with_constraints
model_name_c3 = model_name + '_c3'
experiment_dir_c3 = experiment_dir + '_c3'
model_c3, networkx_graph_c3, variable_name_dict = run_deci_train_with_constraints(
    df_train_cohort, constraint_matrix, model_name_c3, experiment_dir_c3, variables_json_path, device)

Saving logs to bogovirus/runs/bogo_2022_09_27_181158_c3/train_output/summary
Auglag Step: 0
LR: 0.001
Inner Step: 100, loss: 5085.32, log p(x|A): -5085.32, dag: 0.78222778, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 4.01e-06, cts_mse_icgnn: 1.11e+04
Inner Step: 200, loss: 3162.36, log p(x|A): -3162.36, dag: 0.92940804, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 5.2e-06, cts_mse_icgnn: 8.1e+03
Inner Step: 300, loss: 1925.73, log p(x|A): -1925.73, dag: 0.79504959, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 3.95e-06, cts_mse_icgnn: 5.57e+03
Inner Step: 400, loss: 1190.78, log p(x|A): -1190.78, dag: 1.11149728, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec: 0.000, penalty_dag_weighed: 7.01e-06, cts_mse_icgnn: 3.76e+03
Inner Step: 500, loss: 762.53, log p(x|A): -762.53, dag: 1.14675684, log p(A)_sp: -0.00, log q(A): -0.001, H filled: 0.000, rec:

In [ ]:
from helpers.deci_utils import save_deci_unweighted_graph, calc_weighted_graph, save_edges_to_excel, save_deci_weighted_graph

save_deci_unweighted_graph(model_c3, variable_name_dict, experiment_dir_c3)
deci_weighted_graph_c3 = calc_weighted_graph(model_c3, df_train_cohort, variables_json_path, experiment_dir_c3, samples=200)
save_edges_to_excel(deci_weighted_graph_c3, variable_name_dict, experiment_dir_c3)
save_deci_weighted_graph(deci_weighted_graph_c3, variable_name_dict, experiment_dir_c3, conf_threshold=0.7)


In [ ]:
population=5000 #everyone
df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() 

#  setup
df_prev = df_core[df_train_cohort.columns] # sample instead
num_of_days=60
dose = 0.4

df04_c3=run_simsim(model_c2, population, df_core, df_prev, num_of_days, dose)
df04_c3.to_csv(os.path.join(experiment_dir_c3, 'simsim_data', 'deci_constrainted_dose0.4_max60days' +'.csv'))

print(df04_c3['die'].sum())
print(df04_c3['recover'].sum())

In [ ]:
print(df04_c3['die'].sum())
print(df04_c3['recover'].sum())

In [ ]:
population=5000 #everyone
df_core = df_train[df_train.day_number==0].sample(population, random_state=42).copy() 

#  setup
df_prev = df_core[df_train_cohort.columns] # sample instead
num_of_days=60
dose = 1.4

df14_c3=run_simsim(model_c3, population, df_core, df_prev, num_of_days, dose)
df14_c3.to_csv(os.path.join(experiment_dir_c3, 'simsim_data', 'deci_constrainted_dose1.4_max60days' +'.csv'))

print(df14_c3['die'].sum())
print(df14_c3['recover'].sum())

### Concluding graph discovery
The DECI model from Causica offers us a way to discover a causal graph from observational data. The learned graph can be iteratively refined by adding new graph constraints and retraining DECI to obtain a more realistic graph. 

### Average treatment effect
DECI estimates the total ATE rather than the directed effect.

In [ ]:
outcome = 'severity_next'
outcome_idx = dataset.variables.name_to_idx[outcome]
treatment = 'drug'
treatment_idx = dataset.variables.name_to_idx[treatment]

dose_of_drug = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

death_estimated_ate = {}


for dose in dose_of_drug:
    ate = model.cate(
        intervention_idxs=np.array([treatment_idx]),
        intervention_values=np.array([dose]),
        reference_values=np.array([0]),
        Nsamples_per_graph=25
    )
    death_estimated_ate[dose] = ate[0][outcome_idx] #* model.data_processor._cts_normalizers[1].scale_[-1]

ATE using Model_c2 

In [ ]:
outcome = 'severity_next'
outcome_idx = dataset.variables.name_to_idx[outcome]
treatment = 'drug'
treatment_idx = dataset.variables.name_to_idx[treatment]

dose_of_drug = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

death_estimated_ate = {}


for dose in dose_of_drug:
    ate = model_c2.cate(
        intervention_idxs=np.array([treatment_idx]),
        intervention_values=np.array([dose]),
        reference_values=np.array([0]),
        Nsamples_per_graph=25
    )
    death_estimated_ate[dose] = ate[0][outcome_idx] #* model.data_processor._cts_normalizers[1].scale_[-1]

In [ ]:
death_estimated_ate

ATE using model_c3

In [ ]:
outcome = 'severity_next'
outcome_idx = dataset.variables.name_to_idx[outcome]
treatment = 'drug'
treatment_idx = dataset.variables.name_to_idx[treatment]

dose_of_drug = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

death_estimated_ate = {}


for dose in dose_of_drug:
    ate = model_c3.cate(
        intervention_idxs=np.array([treatment_idx]),
        intervention_values=np.array([dose]),
        reference_values=np.array([0]),
        Nsamples_per_graph=25
    )
    death_estimated_ate[dose] = ate[0][outcome_idx] #* model.data_processor._cts_normalizers[1].scale_[-1]

death_estimated_ate

### Conditional average treatment effect

Let's repeat this analysis, but conditional on the patient severity. DECI can estimate the conditional average treatment effect for such a patient.

In [ ]:
df_train['severity'].hist()

High Severity

In [ ]:
# We need to convert the conditioning value correctly
condition_idx = dataset.variables.name_to_idx["severity"]
condition_value = np.zeros((1, model.processed_dim_all))
condition_value[0, condition_idx] = 100.0
condition_value = model.data_processor.process_data(condition_value)
condition_value = condition_value[0, [condition_idx]]

In [ ]:
outcome = 'severity_next'
outcome_idx = dataset.variables.name_to_idx[outcome]
treatment = 'drug'
treatment_idx = dataset.variables.name_to_idx[treatment]

dose_of_drug = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

death_estimated_cate = {}


for dose in dose_of_drug:
    cate_ = model_c2.cate(
        intervention_idxs=np.array([treatment_idx]),
        intervention_values=np.array([dose]),
        reference_values=np.array([0]),
        conditioning_idxs=np.array([condition_idx]),
        conditioning_values=condition_value,
        effect_idxs=np.array([outcome_idx]),
        Nsamples_per_graph=25
    )
    death_estimated_cate[dose] =  cate_[0][0] #* model.data_processor._cts_normalizers[1].scale_[-1]
    print(death_estimated_cate[dose])

death_estimated_cate

Medium Severity

In [ ]:
# We need to convert the conditioning value correctly
condition_idx = dataset.variables.name_to_idx["severity"]
condition_value = np.zeros((1, model.processed_dim_all))
condition_value[0, condition_idx] = 40.0
condition_value = model.data_processor.process_data(condition_value)
condition_value = condition_value[0, [condition_idx]]

In [ ]:
outcome = 'severity_next'
outcome_idx = dataset.variables.name_to_idx[outcome]
treatment = 'drug'
treatment_idx = dataset.variables.name_to_idx[treatment]

dose_of_drug = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

death_estimated_cate = {}


for dose in dose_of_drug:
    cate_ = model_c2.cate(
        intervention_idxs=np.array([treatment_idx]),
        intervention_values=np.array([dose]),
        reference_values=np.array([0]),
        conditioning_idxs=np.array([condition_idx]),
        conditioning_values=condition_value,
        effect_idxs=np.array([outcome_idx]),
        Nsamples_per_graph=25
    )
    death_estimated_cate[dose] =  cate_[0][0] #* model.data_processor._cts_normalizers[1].scale_[-1]
    print(death_estimated_cate[dose])

death_estimated_cate

Low Severity

In [ ]:
# We need to convert the conditioning value correctly
condition_idx = dataset.variables.name_to_idx["severity"]
condition_value = np.zeros((1, model.processed_dim_all))
condition_value[0, condition_idx] = 20.0
condition_value = model.data_processor.process_data(condition_value)
condition_value = condition_value[0, [condition_idx]]

In [ ]:
outcome = 'severity_next'
outcome_idx = dataset.variables.name_to_idx[outcome]
treatment = 'drug'
treatment_idx = dataset.variables.name_to_idx[treatment]

dose_of_drug = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4]

death_estimated_cate = {}


for dose in dose_of_drug:
    cate_ = model_c2.cate(
        intervention_idxs=np.array([treatment_idx]),
        intervention_values=np.array([dose]),
        reference_values=np.array([0]),
        conditioning_idxs=np.array([condition_idx]),
        conditioning_values=condition_value,
        effect_idxs=np.array([outcome_idx]),
        Nsamples_per_graph=25
    )
    death_estimated_cate[dose] =  cate_[0][0] #* model.data_processor._cts_normalizers[1].scale_[-1]
    print(death_estimated_cate[dose])

death_estimated_cate